### Basic import

In [1]:
from pretrainedModel import *
from faceRecognition import *
import pandas as pd
from math import *
from matplotlib import pyplot as plt

### FaceNet pre-trained model

In [2]:
sess, embeddings, images_in, phase_train_in, pnet, rnet, onet = detectionNetwork()

loading the detection/alignment network...
done!
loading the embedding network...
done!


### Set up face DB

In [3]:
# Path of the references faces that will construct the database of faces
pathsKnown = ["/home/sabrine/Pictures/laurene1.jpg",
              "/home/sabrine/Pictures/laurene2.jpg",
              "/home/sabrine/Pictures/laurene3.jpg",
              "/home/sabrine/Pictures/anthony1.jpg",
              "/home/sabrine/Pictures/anthony2.jpg",
              "/home/sabrine/Pictures/anthony3.jpg",
              "/home/sabrine/Pictures/wissem1.jpg",
              "/home/sabrine/Pictures/wissem2.jpg",
              "/home/sabrine/Pictures/wissem3.jpg",
              "/home/sabrine/Pictures/cynthiat1.jpeg",
              "/home/sabrine/Pictures/cynthiat2.jpeg",
              "/home/sabrine/Pictures/cynthiat3.jpeg",
              "/home/sabrine/Pictures/sabrineb1.jpg",
              "/home/sabrine/Pictures/sabrineb2.jpg",
              "/home/sabrine/Pictures/sabrineb3.jpg"]

# Creation of identities names
names = ['Laurene', 'Laurene', 'Laurene', 'Anthony', 'Anthony', 'Anthony', 'Wissem', 'Wissem', 'Wissem',
         'Cynthia', 'Cynthia', 'Cynthia','Sabrine', 'Sabrine', 'Sabrine']

In [4]:
# detection parameters
minsize = 40 
threshold = [ 0.75, 0.75, 0.75 ]
factor = 0.709

In [5]:
df = embeddingKnownFaces(pathsKnown, names, sess, embeddings, images_in,
                   phase_train_in, pnet, rnet, onet,
                   minsize, threshold, factor)

detecting faces...
/home/sabrine/Pictures/laurene1.jpg
(10, 1)
/home/sabrine/Pictures/laurene2.jpg
(10, 1)
/home/sabrine/Pictures/laurene3.jpg
(10, 1)
/home/sabrine/Pictures/anthony1.jpg
(10, 1)
/home/sabrine/Pictures/anthony2.jpg
(10, 1)
/home/sabrine/Pictures/anthony3.jpg
(10, 1)
/home/sabrine/Pictures/wissem1.jpg
(10, 1)
/home/sabrine/Pictures/wissem2.jpg
(10, 1)
/home/sabrine/Pictures/wissem3.jpg
(10, 1)
/home/sabrine/Pictures/cynthiat1.jpeg
(10, 1)
/home/sabrine/Pictures/cynthiat2.jpeg
(10, 1)
/home/sabrine/Pictures/cynthiat3.jpeg
(10, 1)
/home/sabrine/Pictures/sabrineb1.jpg
(10, 1)
/home/sabrine/Pictures/sabrineb2.jpg
(10, 1)
/home/sabrine/Pictures/sabrineb3.jpg
(10, 1)
done!


### Retreive rgbSeq with detection information

In [6]:
synchronize = pd.read_pickle('synchronization/synchronize')

In [44]:
#Not important here, will be done later
#synchronize.trackX = synchronize.trackX.astype(int) * 2
#synchronize.trackY = synchronize.trackY.astype(int) * 2
#synchronize.trackH = synchronize.trackH.astype(int) * 2
#synchronize.trackW = synchronize.trackW.astype(int) * 2

In [9]:
#synchronize.trackX = synchronize.trackX * 2
#synchronize.trackY = synchronize.trackY * 2
#synchronize.trackH = synchronize.trackH * 2
#synchronize.trackW = synchronize.trackW * 2

In [46]:
# Generate a list of name in the same order of the known embeddings without repetition 
uniqueNames = []
for i in names:
    if i not in uniqueNames:
        uniqueNames.append(i)
print (uniqueNames)

['Laurene', 'Anthony', 'Wissem', 'Cynthia', 'Sabrine']


In [32]:
'''
FR for all frame and storing of the returned information in the same order as the df synchronize
'''

bigSmallestArray = []
bigCenters = []
scores = [[] for i in range(len(uniqueNames))]
rgb = []

for seq in synchronize.rgbSeq.unique():
    print seq
    
    #Path of the frames
    path = '/home/sabrine/notebook/framesRGB2/' + str(seq) + '.jpg'
    #path = '/home/sabrine/notebook/framesRGB2/' + '177' + '.jpg'
    
    # List of names, coordinates and distances of the FR process
    fr = faceRecognition(pathsKnown, names, path, sess, embeddings, images_in, phase_train_in, pnet, rnet, onet, df)
    smallestArray = fr[0]
    centers = fr[1]
    
    # check if at least one face has been detected
    if len(fr[0]) > 0:
        for i in range(smallestArray.shape[1]):
            for j in range(len(scores)):
                scores[j].append(smallestArray[j, i])
            bigCenters.append(centers[i])
            rgb.append(seq)
    else:
        # So the index of rgb, bigCenters and scores will be the same as synchronize.rgbSeq.unique()
        # It will simplify the join process of these 2 df
        rgb.append(np.nan)
        bigCenters.append(np.nan)
        for j in range(len(scores)):
                scores[j].append(np.nan)
    
    print '*************'

42
*************
45
*************
47
*************
48
*************
51
Blurry
*************
52
Blurry
*************
53
*************
54
*************
56
*************
58
*************
61
*************
63
*************
65
*************
67
*************
70
*************
71
*************
72
*************
74
*************
75
*************
76
*************
78
*************
79
*************
82
*************
96
*************
98
*************
99
*************
100
*************
102
*************
103
*************
104
*************
105
*************
106
*************
109
*************
110
*************
111
*************
115
*************
117
*************
118
*************
119
*************
120
*************
121
*************
123
*************
124
*************
126
*************
127
*************
128
*************
129
*************
131
*************
133
*************
134
*************
137
*************
141
*************
142
*************
144
*************
145
*************
149
*************
151


#### Check if all list has the same size 

In [33]:
type(centers)

list

In [34]:
len(bigCenters)

184

In [35]:
len(rgb)

184

In [36]:
len(scores[1])

184

#### Creation of the df faces

In [37]:
data = np.asarray(scores).T

In [38]:
faces = pd.DataFrame(data=data)

In [39]:
faces.columns = uniqueNames

In [40]:
faces

,Laurene,Anthony,Wissem,Cynthia,Sabrine
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,1.211830,1.292989,1.168803,1.270894,1.328637
3,1.158829,1.242249,1.080242,1.210182,1.315757
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,1.261341,1.332883,1.259527,1.354414,1.299897


In [41]:
faces['centers'] = bigCenters
faces['rgbSeq'] = rgb

In [42]:
faces

,Laurene,Anthony,Wissem,Cynthia,Sabrine,centers,rgbSeq
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.211830,1.292989,1.168803,1.270894,1.328637,"[1039, 57]",47.0
3,1.158829,1.242249,1.080242,1.210182,1.315757,"[1043, 60]",48.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.261341,1.332883,1.259527,1.354414,1.299897,"[1828, 508]",58.0


In [48]:
# save the result in pickle algorithm
faces.to_pickle('faces')